##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrar el entrenamiento multitrabajador con CPU/GPU

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/multi_worker_cpu_gpu_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

Esta guía demuestra cómo migrar su flujo de trabajo de entrenamiento distribuido multitrabajador de TensorFlow 1 a TensorFlow 2.

Para realizar un entrenamiento multitrabajador con CPUs/GPUs:

- En TensorFlow 1, usted usa tradicionalmente las APIs `tf.estimator.train_and_evaluate` y `tf.estimator.Estimator`.
- En TensorFlow 2, use las API de Keras para escribir el modelo, la función de pérdida, el optimizador y las métricas. Luego, distribuya el entrenamiento con la API `Model.fit` de Keras o un bucle de entrenamiento personalizado (con `tf.GradientTape`) entre múltiples trabajadores con `tf.distribute.experimental.ParameterServerStrategy` o `tf.distribute.MultiWorkerMirroredStrategy`. Para más detalles, consulte los siguientes tutoriales:
    - [Entrenamiento distribuido con TensorFlow](../../guide/distributed_training.ipynb)
    - [Entrenamiento del servidor de parámetros con Model.fit de Keras/un bucle de entrenamiento personalizado](../../tutorials/distribute/parameter_server_training.ipynb)
    - [MultiWorkerMirroredStrategy con Model.fit de Keras](../../tutorials/distribute/multi_worker_with_keras.ipynb)
    - [MultiWorkerMirroredStrategy con un bucle de entrenamiento personalizado](../../tutorials/distribute/multi_worker_with_ctl.ipynb).

## Preparación

Comience con algunas importaciones necesarias y un conjunto de datos sencillo a modo de demostración:

In [ ]:
# The notebook uses a dataset instance for `Model.fit` with
# `ParameterServerStrategy`, which depends on symbols in TF 2.7.
# Install a utility needed for this demonstration
!pip install portpicker

import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

Necesitará la variable de entorno de configuración `'TF_CONFIG'` para entrenar en múltiples máquinas en TensorFlow. Use `'TF_CONFIG'` para especificar las direcciones del `'cluster'` y de las `'task'`s. (Más información en la guía [Entrenamiento distribuido](../...guide/distributed_training.ipynb)).

In [ ]:
import json
import os

tf_config = {
    'cluster': {
        'chief': ['localhost:11111'],
        'worker': ['localhost:12345', 'localhost:23456', 'localhost:21212'],
        'ps': ['localhost:12121', 'localhost:13131'],
    },
    'task': {'type': 'chief', 'index': 0}
}

os.environ['TF_CONFIG'] = json.dumps(tf_config)

Nota: Desafortunadamente, ya que el entrenamiento multitrabajador con las APIs `tf.estimator` en TensorFlow 1 requiere múltiples clientes (lo que sería especialmente difícil de hacer aquí en este bloc de notas Colab), hará que el bloc de notas sea ejecutable sin una variable de entorno `'TF_CONFIG'`, así que vuelve a caer en el entrenamiento local (más información en la sección <em data-md-type="emphasis">Cómo establecer la variable de entorno `'TF_CONFIG'`</em> de la guía [Entrenamiento distribuido con TensorFlow](../../guide/distributed_training.ipynb)).

Use la sentencia `del` para eliminar la variable (pero en el entrenamiento multitrabajador del mundo real en TensorFlow 1, no tendrá que hacerlo):

In [ ]:
del os.environ['TF_CONFIG']

## TensorFlow 1: Entrenamiento distribuido multitrabajador con APIs tf.estimator

El siguiente fragmento de código demuestra el flujo de trabajo canónico del entrenamiento multitrabajador en TF1: usará un `tf.estimator.Estimator`, un `tf.estimator.TrainSpec`, un `tf.estimator.EvalSpec`, y la API `tf.estimator.train_and_evaluate` para distribuir el entrenamiento:

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)
train_spec = tf1.estimator.TrainSpec(input_fn=_input_fn)
eval_spec = tf1.estimator.EvalSpec(input_fn=_eval_input_fn)
tf1.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## TensorFlow 2: Entrenamiento multitrabajador con estrategias de distribución

En TensorFlow 2, el entrenamiento distribuido entre varios trabajadores con CPU, GPU y TPU se realiza a través de `tf.distribute.Strategy`s.

El siguiente ejemplo demuestra cómo usar dos estrategias de este tipo: `tf.distribute.experimental.ParameterServerStrategy` y `tf.distribute.MultiWorkerMirroredStrategy`, ambas diseñadas para el entrenamiento de CPU/GPU con múltiples trabajadores.

`ParameterServerStrategy` emplea un *coordinador* (`'chiel'`), que lo hace más amigable con el entorno de este bloc de notas Colab. Aquí usará algunas utilidades para configurar los elementos de apoyo esenciales para una experiencia ejecutable: creará un *cluster interno del proceso*, en el que se usarán hilos para simular los servidores de parámetros (`'ps'`) y los trabajadores (`'worker'`). Para más información sobre el entrenamiento de servidores de parámetros, consulte el tutorial [Entrenamiento de servidores de parámetros con ParameterServerStrategy](../../tutorials/distribute/parameter_server_training.ipynb).

En este ejemplo, defina primero la variable de entorno `'TF_CONFIG'` con un `tf.distribute.cluster_resolver.TFConfigClusterResolver` para ofrecer la información del clúster. Si utiliza un sistema de gestión de clústeres para su entrenamiento distribuido, compruebe si ya suministra `'TF_CONFIG'` para usted, en cuyo caso no necesitará establecer explícitamente esta variable de entorno (más información en la sección <em data-md-type="emphasis">Cómo configurar la variable de entorno `'TF_CONFIG'`</em> de la guía [Entrenamiento distribuido con TensorFlow](../../guide/distributed_training.ipynb)).

In [ ]:
# Find ports that are available for the `'chief'` (the coordinator),
# `'worker'`s, and `'ps'` (parameter servers).
import portpicker

chief_port = portpicker.pick_unused_port()
worker_ports = [portpicker.pick_unused_port() for _ in range(3)]
ps_ports = [portpicker.pick_unused_port() for _ in range(2)]

# Dump the cluster information to `'TF_CONFIG'`.
tf_config = {
    'cluster': {
        'chief': ["localhost:%s" % chief_port],
        'worker': ["localhost:%s" % port for port in worker_ports],
        'ps':  ["localhost:%s" % port for port in ps_ports],
    },
    'task': {'type': 'chief', 'index': 0}
}
os.environ['TF_CONFIG'] = json.dumps(tf_config)

# Use a cluster resolver to bridge the information to the strategy created below.
cluster_resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()

Después, cree `tf.distribute.Server`s para los trabajadores y servidores de parámetros uno a uno:

In [ ]:
# Workers need some inter_ops threads to work properly.
# This is only needed for this notebook to demo. Real servers
# should not need this.
worker_config = tf.compat.v1.ConfigProto()
worker_config.inter_op_parallelism_threads = 4

for i in range(3):
  tf.distribute.Server(
      cluster_resolver.cluster_spec(),
      job_name="worker",
      task_index=i,
      config=worker_config)

for i in range(2):
  tf.distribute.Server(
      cluster_resolver.cluster_spec(),
      job_name="ps",
      task_index=i)

En el entrenamiento distribuido en el mundo real, en lugar de iniciar todos los `tf.distribute.Server`s en el coordinador, usará múltiples máquinas, y las que se designen como `"worker"`s y `"ps"` (servidores de parámetros) ejecutarán cada una un `tf.distribute.Server`. Consulte la sección *Clusters en el mundo real* del tutorial [Servidor de parámetros](../../tutorials/distribute/parameter_server_training.ipynb) para obtener más detalles.

Con todo listo, cree el objeto `ParameterServerStrategy`:

In [ ]:
strategy = tf.distribute.experimental.ParameterServerStrategy(cluster_resolver)

Una vez que haya creado un objeto de estrategia, defina el modelo, el optimizador y otras variables, y llame al `Model.compile` de Keras dentro de la API `Strategy.scope` para distribuir el entrenamiento (consulte la documentación de la API `Strategy.scope` para obtener más información).

Si prefiere personalizar su entrenamiento definiendo, por ejemplo, las pasadas hacia delante y hacia atrás, consulte la sección *Entrenamiento con un bucle de entrenamiento personalizado* del tutorial [Entrenamiento de servidor de parámetros](../../tutorials/distribute/parameter_server_training.ipynb) para saber más.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
      (features, labels)).shuffle(10).repeat().batch(64)

eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).repeat().batch(1)

with strategy.scope():
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
  optimizer = tf.keras.optimizers.legacy.Adagrad(learning_rate=0.05)
  model.compile(optimizer, "mse")

model.fit(dataset, epochs=5, steps_per_epoch=10)

In [ ]:
model.evaluate(eval_dataset, steps=10, return_dict=True)

> **Particionadores (`tf.distribute.experimental.partitioners`)**
>
> `ParameterServerStrategy` en TensorFlow 2 es compatible con la partición de variables y ofrece los mismos particionadores que TensorFlow 1, con nombres menos confusos:
>
> - `tf.compat.v1.variable_axis_size_partitioner` -&gt; `tf.distribute.experimental.partitioners.MaxSizePartitioner`: un particionador que conserva los fragmentos por debajo de un tamaño máximo).
> - `tf.compat.v1.min_max_variable_partitioner` -&gt; `tf.distribute.experimental.partitioners.MinSizePartitioner`: un particionador que asigna un tamaño mínimo por fragmento.
> - `tf.compat.v1.fixed_size_partitioner` -&gt; `tf.distribute.experimental.partitioners.FixedShardsPartitioner`: un particionador que asigna un número fijo de fragmentos.

Como alternativa, puede usar un objeto `MultiWorkerMirroredStrategy`:

In [ ]:
# To clean up the `TF_CONFIG` used for `ParameterServerStrategy`.
del os.environ['TF_CONFIG']
strategy = tf.distribute.MultiWorkerMirroredStrategy()

Puede reemplazar la estrategia utilizada anteriormente por un objeto `MultiWorkerMirroredStrategy` para realizar el entrenamiento con esta estrategia.

Al igual que con las APIs `tf.estimator`, dado que `MultiWorkerMirroredStrategy` es una estrategia multicliente, no existe una forma sencilla de ejecutar un entrenamiento distribuido en este bloc de notas Colab. Por lo tanto, si se sustituye el código anterior por esta estrategia, se acabará ejecutando todo localmente. Los tutoriales Entrenamiento multitrabajador [con Model.fit de Keras](../../tutorials/distribute/multi_worker_with_keras.ipynb)/[un bucle de entrenamiento personalizado](../../tutorials/distribute/multi_worker_with_ctl.ipynb) demuestran cómo ejecutar un entrenamiento multitrabajador con la variable `'TF_CONFIG'` configurada, con dos trabajadores en un localhost en Colab. En la práctica, usted crearía múltiples trabajadores en direcciones IP/puertos externos, y usaría la variable `'TF_CONFIG'` para especificar la configuración del cluster para cada trabajador.

## Siguientes pasos

Para saber más sobre el entrenamiento distribuido multitrabajador con `tf.distribute.experimental.ParameterServerStrategy` y `tf.distribute.MultiWorkerMirroredStrategy` en TensorFlow 2, tenga en cuenta los siguientes recursos:

- Tutorial: [Entrenamiento del servidor de parámetros con ParameterServerStrategy y Model.fit de Keras/bucle de entrenamiento personalizado](../../tutorials/distribute/parameter_server_training.ipynb)
- Tutorial: [Entrenamiento multitrabajador con MultiWorkerMirroredStrategy y Model.fit de Keras](../../tutorials/distribute/multi_worker_with_keras.ipynb)
- Tutorial: [Entrenamiento multitrabajador con MultiWorkerMirroredStrategy y un bucle de entrenamiento personalizado](../../tutorials/distribute/multi_worker_with_ctl.ipynb)
- Guía: [Entrenamiento distribuido con TensorFlow](../../guide/distributed_training.ipynb)
- Guía: [Optimice el rendimiento de la GPU de TensorFlow con TensorFlow Profiler](../../guide/gpu_performance_analysis.ipynb)
- Guía: [Cómo utilizar una GPU](../../guide/gpu.ipynb) (la sección Utilizar varias GPU)